In [3]:
#htsf\Scripts\activate    

# Import libraries
import pandas as pd
from plotnine import ggplot, aes, geom_line

import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'src'))
import hyperarch

%matplotlib inline


def get_data(file_loc, verbose=True):
    data = pd.read_csv(file_loc)
    # Update Data Types
    data['date'] = pd.to_datetime(data['date'])
    data['subcategory'] = data['subcategory'].astype('category')
    data['category'] = data['category'].astype('category')
    if verbose is True:
        display(data.head())
        data.info()
        for col in data.columns:
            if data[col].dtype == 'category':
                print(f'{col} unique values: {len(data[col].unique())}')
            elif data[col].dtype == 'datetime64[ns]':
                print(f'Min Date: {data[col].min()}')
                print(f'Max Date: {data[col].max()}')
            else:
                print(f'{col}:\n{data[col].describe()}\n')
    return data

# Read data
h_df = get_data('../data/hierarchal_agg.csv', verbose = True)
g_df = get_data('../data/grouped_agg.csv', verbose = False)

,date,category,subcategory,value
0,2021-01-01,category 1,subcat 1,1.0
1,2021-02-01,category 1,subcat 1,2.0
2,2021-03-01,category 1,subcat 1,3.0
3,2021-04-01,category 1,subcat 1,3.3
4,2021-05-01,category 1,subcat 1,5.4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         120 non-null    datetime64[ns]
 1   category     120 non-null    category      
 2   subcategory  120 non-null    category      
 3   value        120 non-null    float64       
dtypes: category(2), datetime64[ns](1), float64(1)
memory usage: 2.6 KB
Min Date: 2021-01-01 00:00:00
Max Date: 2022-12-01 00:00:00
category unique values: 2
subcategory unique values: 5
value:
count    120.000000
mean       6.240000
std        5.328556
min        0.000000
25%        2.000000
50%        5.000000
75%        9.000000
max       25.000000
Name: value, dtype: float64



In [ ]:


# Grouped hierarchy
hierarchy_df, h_bottom, h_labels = hyperarch.get_hierarchal(h_df, 'category', 'subcategory', agg_type='hierarchy')   
h_s = hyperarch.get_S(h_bottom, h_labels, agg_type='hierarchy')
h_models = hyperarch.get_models(hierarchy_df, steps_out=4)
h_yhat = hyperarch.get_forecast_matrix(h_models)
h_reconciled_yhat = hyperarch.reconcile(h_yhat, h_s, method='ols')

# Grouped hierarchy
grouped_df, g_bottom, g_labels = hyperarch.get_hierarchal(g_df, 'category', 'subcategory', agg_type='grouped')   
g_s = hyperarch.get_S(g_bottom, g_labels, agg_type='grouped')
g_models = hyperarch.get_models(grouped_df, steps_out=4)
g_yhat = hyperarch.get_forecast_matrix(g_models)
g_reconciled_yhat = hyperarch.reconcile(g_yhat, g_s, method='ols')